# Lecture 2: Computational Thinking with Python

## Key Concept Review

### String operations

Before we begin the exercises, let's review some essential string operations you'll need:

#### Looping Over Strings
You can loop through each character in a string using a for loop:

In [1]:
formula = "H2O"
for character in formula:
    print(character)

H
2
O


#### Useful String Methods

```python
string.isalpha() - returns True if the character is a letter
string.isdigit() - returns True if the character is a number
string.upper() - converts to uppercase
string.lower() - converts to lowercase
```

Get familiar with these functions by experimenting with them. E.g. what is the output of the following?
```python
"H".isalpha()
```

In [2]:
"H".isalpha()

True

### String Indexing and Slicing
```python
formula = "CH4"
print(formula[0])    # Output: "C" (first character)
print(formula[1:3])  # Output: "H4" (characters from index 1 to 2)
print(len(formula))  # Output: 3 (length of string)
```
Familiarise yourself with these functions.


In [3]:
formula = "CH4"
print(formula[0])    # Output: "C" (first character)
print(formula[1:3])  # Output: "H4" (characters from index 1 to 2)
print(len(formula))  # Output: 3 (length of string)

C
H4
3


### Converting Strings to Numbers
```python
number_string = "42"
number = int(number_string)  # Converts "42" to integer 42
```

Familiarise yourself with these functions.

In [4]:
number_string = "42"
number = int(number_string)  # Converts "42" to integer 42
type(number)

int

## Exercise 1: Molecular Formula Parser

**Objective:** Write a function `parse_molecular_formula(formula)` that counts atoms in molecular formulas. Start with the simple cases, but design your solution to handle increasingly complex inputs without major rewrites.

**Think computationally:** How can you break this problem into smaller, reusable pieces that won't need to be completely rewritten when the requirements change?


Strategy:
- Define a function to count the number of atoms of an element in a formula; if no number is written, assign the value 1 and otherwise assign the number written.
- Identify a pattern that finds elements and numbers of atoms (if given) in formulas using RegEx (https://regex101.com/).
- Initialize a dictionary to which we can add instances of atoms and their number
- Iterate over the RegEx-defined patterns in the formula to fill the dictionary

In [ ]:
import re # This is needed in order to use RegEx (regular expressions)

def convert_quantities(quantity : str):
    if len(quantity) == 0:
        return 1
    else:
        return int(quantity)

def parse_molecular_formula(formula: str) -> dict[str, int]:
    """
    Because there is "N", "I" and "Ni" in elements, to split this in a unique way we require the correct capitalization in the input.
    This works for sum formulas only.
    """
    atom_dict = dict()
    pattern = r"([A-Z][a-z]?)([0-9]*)"

    components = re.findall(pattern, formula)
    for component in components:
        atom_dict[component[0]] = convert_quantities(component[1])

    return atom_dict

In [1]:
# Second option
# Requires import string (and import Dict), does not require import re

import string
from typing import Dict
    
def parse_molecular_formula(formula: str) -> Dict[str, int]:
    """
    Because there is "N", "I" and "Ni" in the elements to split this in a unique way we require the correct capitalization in the input.
    """
    atom_dict = dict()
    
    tokens = []
    token = ""
    
    for ch in formula:
        if ch in string.ascii_uppercase:
            if len(token) > 0:
                tokens.append(token)
            token = ch
        else:
            token += ch

    if len(token) > 0:
        tokens.append(token)    
    
    for current in tokens:
        digits = "".join(dg for dg in current if dg in string.digits)
        element = "".join(ch for ch in current if ch not in string.digits)
        
        if len(digits) == 0:
            count = 1
        else:
            count = int(digits)
        atom_dict[element] = count

    return atom_dict

In [2]:
print(parse_molecular_formula("H2O"))
print(parse_molecular_formula("CaCl2"))
print(parse_molecular_formula("C12H22O11"))
print(parse_molecular_formula("C2H5OH"))

{'H': 2, 'O': 1}
{'Ca': 1, 'Cl': 2}
{'C': 12, 'H': 22, 'O': 11}
{'C': 2, 'H': 1, 'O': 1}


The function as defined above fails for C2H5OH because the new "H" entry overwrites the former "H" entry.

Revised strategy: Initiate a dictionary that has default entries so that each atom does not need to be assigned as a new entry (thereby overwriting previous instances of that atom as they have the same key), but rather added to the dictionary so that instances of atoms are summed.

In [7]:
import re # This is needed in order to use RegEx (regular expressions)
from collections import defaultdict #defaultdict creates default entries and thus allows to add to existing entries

def convert_quantities(quantity : str):
    if len(quantity) == 0:
        return 1
    else:
        return int(quantity)

def parse_molecular_formula(formula: str) -> dict[str, int]:
    """
    Because there is "N", "I" and "Ni" in elements, to split this in a unique way we require the correct capitalization in the input.
    """
    atom_dict = defaultdict(int)
    pattern = r"([A-Z][a-z]?)([0-9]*)"

    components = re.findall(pattern, formula)
    for component in components:
        atom_dict[component[0]] += convert_quantities(component[1])

    return atom_dict

# Alternative solution using Counter instead -- requires from collections import Counter
# Not shorter than the solution with defaultdict, and the code is a bit more complex, so not preferred

# def parse_molecular_formula(formula: str) -> dict[str, int]:
#     """
#     Because there is "N", "I" and "Ni" in elements, to split this in a unique way we require the correct capitalization in the input.
#     """
#     atom_counter = Counter()
#     pattern = r"([A-Z][a-z]?)([0-9]*)"

#     components = re.findall(pattern, formula)
#     for component in components:
#         atom_counter += Counter({component[0]:convert_quantities(component[1])})

#    return atom_counter
    
parse_molecular_formula("C2H5OH")

defaultdict(int, {'C': 2, 'H': 6, 'O': 1})

In [ ]:
# Alternative Double while loop with two pointers (i, j): 
# While 1: Loop over pairs of atom/element e.g. Cl2
# While 2: Loop over atom/element pair and extract element and number of atoms

from typing import Dict

def parse_molecular_formula(formula: str) -> Dict[str, int]:
    atom_dict = dict()

    i = 0
    while i < len(formula):
        extracted_element = ""
        extracted_n_atoms = ""

        j = 0
        while (j+i) < len(formula):
            # Break condition: Start of next element
            if extracted_element != "" and formula[i+j].lower() != formula[i+j]: # Alternative: extracted_element != "" and formula[i + j].isupper()
                break
            elif formula[i+j].isalpha():
                extracted_element += formula[i+j]
            elif formula[i+j].isnumeric():
                extracted_n_atoms += formula[i+j]
            j += 1
        
        if extracted_element in atom_dict:
            atom_dict[extracted_element] = atom_dict[extracted_element] + (int(extracted_n_atoms) if extracted_n_atoms != "" else 1)
        else:
            atom_dict[extracted_element] = int(extracted_n_atoms) if extracted_n_atoms != "" else 1 
        
        i = i + j
    
    return atom_dict

### Test cases: Set 1 Basic formulae

The test cases will increase in difficulty with each set. Start with the first set, but think about how your approach will scale to the later, more complex cases.

In [3]:
# Simple single-letter elements, single-digit counts
test_cases_level1 = [
    ("H2O", {"H": 2, "O": 1}),
    ("CH4", {"C": 1, "H": 4}),
    ("CO2", {"C": 1, "O": 2}),
    ("NH3", {"N": 1, "H": 3})
]

for formula, expected in test_cases_level1:
    result = parse_molecular_formula(formula)
    print(f"{formula}: {result} {'✓' if result == expected else '✗'}")

H2O: {'H': 2, 'O': 1} ✓
CH4: {'C': 1, 'H': 4} ✓
CO2: {'C': 1, 'O': 2} ✓
NH3: {'N': 1, 'H': 3} ✓


### Test cases: Set 2 Multi-letter Elements

In [4]:
# Two-letter elements like Ca, Cl, Na
test_cases_level2 = [
    ("NaCl", {"Na": 1, "Cl": 1}),
    ("CaCl2", {"Ca": 1, "Cl": 2}),
    ("MgO", {"Mg": 1, "O": 1}),
    ("AlCl3", {"Al": 1, "Cl": 3}),
    ("H2SO4", {"H": 2, "S": 1, "O": 4})
]

for formula, expected in test_cases_level2:
    result = parse_molecular_formula(formula)
    print(f"{formula}: {result} {'✓' if result == expected else '✗'}")


NaCl: {'Na': 1, 'Cl': 1} ✓
CaCl2: {'Ca': 1, 'Cl': 2} ✓
MgO: {'Mg': 1, 'O': 1} ✓
AlCl3: {'Al': 1, 'Cl': 3} ✓
H2SO4: {'H': 2, 'S': 1, 'O': 4} ✓


### Test cases: Set 3 Large Numbers

In [5]:
# Multi-digit counts
test_cases_level3 = [
    ("C12H22O11", {"C": 12, "H": 22, "O": 11}),  # Sucrose
    ("Ca10P6O26", {"Ca": 10, "P": 6, "O": 26}),   # Hydroxyapatite
    ("C2H5OH", {"C": 2, "H": 6, "O": 1}),         # Ethanol
    ("Fe2O3", {"Fe": 2, "O": 3})                  # Iron oxide
]

for formula, expected in test_cases_level3:
    result = parse_molecular_formula(formula)
    print(f"{formula}: {result} {'✓' if result == expected else '✗'}")


C12H22O11: {'C': 12, 'H': 22, 'O': 11} ✓
Ca10P6O26: {'Ca': 10, 'P': 6, 'O': 26} ✓
C2H5OH: {'C': 2, 'H': 1, 'O': 1} ✗
Fe2O3: {'Fe': 2, 'O': 3} ✓
